## Experimentation on CLOI dataset

Create graph dataset from CLOI data and visualize CLOI results

In [ ]:
import math
import pickle
import os
import json
import uuid

import numpy as np
from tqdm import tqdm
import open3d as o3d
from numpy.random import default_rng
from compas.geometry import oriented_bounding_box_numpy
import ifcopenshell.template
import ifcopenshell

from src.geometry import sq_distance, get_dimensions_points, vector_normalise, rot2eul, get_labelcloud_bbox
from src.cloud import refine_cloud

In [ ]:
# constants
data_path = "/mnt/f/datasets/AVEVA_newfacility/AVEVA_newfacility/exports/"
save_path = "../cloi/"
shapes = ['Pipes', 'Flanges', 'Elbows']
labels = ['TUBE', 'FLANGE', 'ELBOW', 'TEE', 'BEND']



In [ ]:
dataset = {}
for i, shape in enumerate(shapes):
    clouds = []
    elements = os.listdir(data_path + shape)

    points = 0
    for el in tqdm(elements):
        cl = refine_cloud(data_path + shape + '/' + el)
        if cl is not None:
            #print(len(cl))
            clouds.append(cl)
            points += len(cl)
    
    print(len(clouds), points)
    dataset[labels[i]] = clouds


In [ ]:
with open(save_path + 'clouds_test.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
with open(save_path + 'clouds_test.pkl', 'rb') as f:
    dataset = pickle.load(f)

### visualize

In [ ]:
# create and visualize a combined cloud
full = []
for label in dataset:
    merged = np.concatenate(dataset[label])
    full.append(merged)
    print(merged.shape)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(merged)
    o3d.io.write_point_cloud(save_path + label + ".ply", pcd)
    
full = np.concatenate(full)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(full)
o3d.io.write_point_cloud(save_path + "combined.ply", pcd)

### graph nodes

In [ ]:
# convert to graph dataset node info, and generate labelcloud annotation
types = ['FLANGE', 'ELBOW', 'TEE', 'TUBE', 'BEND']

def process_nodes(dataset):
    
    nodes = []
    points = []
    bboxes = []
    error_count = 0
    count = 0
    
    for j in dataset:
        print(j)
        for i, el in tqdm(enumerate(dataset[j])):
            try:
                center, lengths, dominant_direction = get_dimensions_points(el)
                node = [types.index(j), center, lengths, dominant_direction, 
                              count]

                bbox = get_labelcloud_bbox(el, j)

                count += 1
                bboxes.append(bbox)
                nodes.append(node)
                points.append(el)

            except Exception as e:
                error_count += 1
                print(len(el))
                print(e)

    print(error_count)
    return([nodes, points], bboxes)


In [ ]:
node_info, bboxes = process_nodes(dataset)

In [ ]:
with open(save_path + 'data.json', 'w') as f:
    json.dump({'objects':bboxes}, f)

In [ ]:
print(len(node_info), len(node_info[0]), len(node_info[1]))

with open(save_path + 'nodes_cloi.pkl', 'wb') as f:
    pickle.dump(node_info, f)

### edges

In [ ]:
# convert labelcloud annotations to graph edges
with open(save_path + 'combined_rels.json', 'r') as f:
    annots = json.load(f)

In [ ]:
edges = [i['bboxes'] for i in annots['relationships']]
with open(save_path + 'edges_cloi.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [ ]:
# convert graph predictions back to labelcloud format for visualization
with open('../eval/cloi_non_rep.pkl', 'rb') as f:
    non_rep_TPs, non_rep_FPs, non_rep_FNs = pickle.load(f)
#print(refined_TPs[10])
prediction_sets = [non_rep_TPs, non_rep_FPs, non_rep_FNs] 

In [ ]:
rel_dict = { "folder": "pointclouds",
            "filename": "combined.ply",
            "path": "pointclouds\\combined.ply",}

relationships = []

for i, preds in enumerate(prediction_sets):
    if i == 0:
        label = 'TP'
    elif i == 1:
        label = 'FP'
    else:
        label = 'FN'
        
    for p in preds:
        rel = {'name': label,
              'bboxes': p.tolist()}
        relationships.append(rel)
        
rel_dict['relationships'] = relationships  

In [ ]:
print(rel_dict)

In [ ]:
with open(save_path + 'cloi_vis.json', 'w') as f:
    json.dump(rel_dict, f)